Background reading: http://mlwave.com/kaggle-ensembling-guide/

In [1]:
import pandas as pd

In [2]:
# Load submission files

"""
gb_model: new feature engineering, 5 features, test accuracy = 0.843, AUC = 0.891
log_model: new feature engineering, 7 features, test accuracy = 0.87, AUC = 0.894 
forest_model: can't remember, old features
forest_model2: feature engineering, 7 features, test accuracy = 0.843, AUC = 0.859
forest_model3: new feature engineering, all features, test accuracy = 0.798, AUC = 0.825

best scoring model on kaggle was an old log_model
"""

log_model = pd.read_csv('submission_logistic.csv')
# forest_model = pd.read_csv('submission_forest.csv')
gb_model = pd.read_csv('submission_gradientboost.csv')
# forest_model2 = pd.read_csv('submission_forest2.csv')
forest_model3 = pd.read_csv('submission_forest3.csv') 

In [12]:
w = 2
# vote_DF = pd.DataFrame({'Forest1': forest_model.Survived, 'Forest2': forest_model2.Survived, \
#                        'GBBoost': gb_model.Survived * w, 'Logistic': log_model.Survived * w, 'Forest3': forest_model3.Survived})

vote_DF = pd.DataFrame({'GBBoost': gb_model.Survived, 'Logistic': log_model.Survived * w, 'Forest3': forest_model3.Survived})

# Model correlations

In [4]:
vote_DF.corr()

,Forest3,GBBoost,Logistic
Forest3,1.000000,0.744119,0.728659
GBBoost,0.744119,1.000000,0.736021
Logistic,0.728659,0.736021,1.000000


# Majority vote

In [13]:
vote_DF[100:150]

,Forest3,GBBoost,Logistic
100,1,1,2
101,0,0,0
102,0,0,0
103,0,0,0
104,0,0,0
105,0,0,0
106,0,0,0
107,0,0,0
108,0,0,0
109,0,0,0


In [31]:
log_model.iloc[141]

PassengerId    1033
Survived          1
Name: 141, dtype: int64

In [6]:
# print(sum(forest_model.Survived), sum(forest_model2.Survived), sum(gb_model.Survived), sum(log_model.Survived), sum(forest_model3.Survived))
print(sum(gb_model.Survived), sum(log_model.Survived), sum(forest_model3.Survived))

149 108 148


In [14]:
vote_series = []
for i in range(len(vote_DF)):
    row_total = sum(vote_DF.iloc[i])
    if row_total < 2:
        vote_series.append(0)
    else:
        vote_series.append(1)

In [15]:
sum(vote_series)

132

In [16]:
sum(vote_series)/len(vote_series)

0.3157894736842105

In [17]:
rowSumList = [sum(vote_DF.iloc[i]) for i in range(len(vote_DF))]

In [27]:
# look at models and see which ones are most frequently in 'alignment' when 3 or more are in agreement
count = 0
logCount = 0
gbCount = 0
forest1Count = 0
forest2Count = 0
forest3Count = 0
for i in range(len(vote_DF)):
    if rowSumList[i] >= 3:
        count += 1
        if vote_DF.Logistic[i] == 1 * w:
            logCount += 1
        if vote_DF.GBBoost[i] == 1 * w:
            gbCount += 1
        if vote_DF.Forest1[i] == 1:
            forest1Count += 1
        if vote_DF.Forest2[i] == 1:
            forest2Count += 1
        if vote_DF.Forest3[i] == 1:
            forest3Count += 1
print('Logistic', count, logCount)
print('GBBoost', count, gbCount)
print('Forest1', count, forest1Count)
print('Forest2', count, forest2Count)
print('Forest3', count, forest3Count)

Logistic 132 107
GBBoost 132 126
Forest1 132 129
Forest2 132 121
Forest3 132 123


In [29]:
# look at models and see which ones are most frequently suggesting survival when other 4 disagree
count = 0
logCount = 0
gbCount = 0
forest1Count = 0
forest2Count = 0
forest3Count = 0
for i in range(len(vote_DF)):
    if rowSumList[i] <= 1.1:
        count += 1
        if vote_DF.Logistic[i] == 1 * w:
            logCount += 1
        if vote_DF.GBBoost[i] == 1 * w:
            gbCount += 1
        if vote_DF.Forest1[i] == 1:
            forest1Count += 1
        if vote_DF.Forest2[i] == 1:
            forest2Count += 1
        if vote_DF.Forest3[i] == 1:
            forest2Count += 1
print('Logistic', count, logCount)
print('GBBoost', count, gbCount)
print('Forest1', count, forest1Count)
print('Forest2', count, forest2Count)
print('Forest3', count, forest3Count)

Logistic 259 0
GBBoost 259 10
Forest1 259 9
Forest2 259 30
Forest3 259 0


# New submission

In [18]:
# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pd.DataFrame({
        "PassengerId": log_model["PassengerId"],
        "Survived": vote_series
    })

# Any files you save will be available in the output tab below
submission.to_csv('submission.csv', index=False)

In [19]:
ensemble1 = pd.read_csv('submission_ensemble.csv')
ensemble2 = pd.read_csv('submission_ensemble3.csv') 

In [20]:
ensembleDF = pd.DataFrame({'ens1': ensemble1.Survived, 'ens2': ensemble2.Survived})

In [21]:
ensembleDF.corr()

,ens1,ens2
ens1,1.000000,0.874555
ens2,0.874555,1.000000


In [23]:
ensembleDF[60:80]

,ens1,ens2
60,0,0
61,0,0
62,0,0
63,1,1
64,1,1
65,1,1
66,1,1
67,0,0
68,0,0
69,1,1
